In [478]:
import yfinance as yf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from itertools import product

In [486]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', 30)

### Optimized Portfolio of Sector Indexes

In [491]:
symbols = ['XLC', 'XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLB', 'XLRE', 'XLK', 'XLU']
data = [
    pd.DataFrame(yf.Ticker(symbol).history(start = '2010-01-01', end = '2024-09-18', interval = '3mo')) # get the data
    .reset_index()[::-4].sort_values(by = 'Date')
    for symbol in symbols
]
cleaned_data = [
   df.set_index(np.arange(15 - df.shape[0], 15, 1)) for df in data 
]
returns = [
    df['Close'].pct_change() # get return series
    for df in cleaned_data
]
means = [
    return_.mean() # get mean return
    for return_ in returns
]
stds = [
    return_.std() # get std dev of returns
    for return_ in returns
]
df_returns = pd.DataFrame(returns).loc[:, np.arange(0, 15, 1)].assign(symbol = symbols).set_index('symbol').T

df = pd.DataFrame()
df['symbol'] = symbols
df['exp_return'] = means
df['std_dev'] = stds
df = df.set_index('symbol')

df_corr = df_returns.corr()
df_corr = df_corr.rename({col: col + '_corr' for col in df_corr.columns}).T

df = pd.concat([df, df_corr], axis = 1)

combs = [i for i in product(symbols, repeat = 2)]

df_covariances = pd.DataFrame()
for symbol in symbols:
    covariances = []
    for comb in combs:
        if comb[0] == symbol:
            std1 = df.loc[comb[0]]['std_dev']
            std2 = df.loc[comb[1]]['std_dev']
            corr = df.loc[comb[0], comb[1] + '_corr']
            cov = std1 * std2 * corr
            covariances.append(cov)
    df_covariances[symbol] = covariances

df_covariances = df_covariances.T.rename(columns = {i: symbol + '_cov' for i, symbol in zip(np.arange(0, 11, 1), symbols)})

df = pd.concat([df, df_covariances], axis = 1)

In [492]:
df

,exp_return,std_dev,XLC_corr,XLY_corr,XLP_corr,XLE_corr,XLF_corr,XLV_corr,XLI_corr,XLB_corr,XLRE_corr,XLK_corr,XLU_corr,XLC_cov,XLY_cov,XLP_cov,XLE_cov,XLF_cov,XLV_cov,XLI_cov,XLB_cov,XLRE_cov,XLK_cov,XLU_cov
XLC,0.153937,0.239113,1.000000,0.867303,0.526211,-0.338174,0.407727,0.538545,0.708741,0.793462,0.439628,0.907765,0.028080,0.057175,0.029612,0.009719,-0.025698,0.022369,0.014504,0.025982,0.028249,0.020293,0.039363,0.000798
XLY,0.156998,0.142789,0.867303,1.000000,0.396426,-0.046211,0.570196,0.794288,0.676836,0.564573,0.402378,0.656923,-0.075665,0.029612,0.020389,0.004372,-0.002097,0.018680,0.012775,0.014817,0.012003,0.011091,0.017011,-0.001284
XLP,0.113593,0.077244,0.526211,0.396426,1.000000,-0.060728,0.448782,0.465498,0.517001,0.525792,0.859273,0.292629,0.710965,0.009719,0.004372,0.005967,-0.001491,0.007954,0.004050,0.006123,0.006047,0.012813,0.004099,0.006528
XLE,0.111662,0.317799,-0.338174,-0.046211,-0.060728,1.000000,0.468635,0.155615,0.363630,0.344220,0.124691,-0.081035,-0.041519,-0.025698,-0.002097,-0.001491,0.100996,0.034171,0.005570,0.017717,0.016288,0.007650,-0.004670,-0.001568
XLF,0.143569,0.229437,0.407727,0.570196,0.448782,0.468635,1.000000,0.655293,0.881981,0.766368,0.735041,0.423145,0.345337,0.022369,0.018680,0.007954,0.034171,0.052642,0.016935,0.031025,0.026181,0.032556,0.017606,0.009418
XLV,0.147989,0.112635,0.538545,0.794288,0.465498,0.155615,0.655293,1.000000,0.733776,0.720974,0.364718,0.587532,0.055759,0.014504,0.012775,0.004050,0.005570,0.016935,0.012687,0.012671,0.012091,0.007930,0.012001,0.000747
XLI,0.139916,0.153314,0.708741,0.676836,0.517001,0.363630,0.881981,0.733776,1.000000,0.895329,0.620081,0.612437,0.237314,0.025982,0.014817,0.006123,0.017717,0.031025,0.012671,0.023505,0.020438,0.018352,0.017028,0.004325
XLB,0.111861,0.148895,0.793462,0.564573,0.525792,0.344220,0.766368,0.720974,0.895329,1.000000,0.555556,0.725971,0.201326,0.028249,0.012003,0.006047,0.016288,0.026181,0.012091,0.020438,0.022170,0.015969,0.019602,0.003563
XLRE,0.091537,0.193044,0.439628,0.402378,0.859273,0.124691,0.735041,0.364718,0.620081,0.555556,1.000000,0.248345,0.784659,0.020293,0.011091,0.012813,0.007650,0.032556,0.007930,0.018352,0.015969,0.037266,0.008694,0.018006
XLK,0.205261,0.181347,0.907765,0.656923,0.292629,-0.081035,0.423145,0.587532,0.612437,0.725971,0.248345,1.000000,-0.075546,0.039363,0.017011,0.004099,-0.004670,0.017606,0.012001,0.017028,0.019602,0.008694,0.032887,-0.001629


### 